In [1]:
import pandas as pd

In [7]:
project_id = Page.objects.get(title='Jamun').id
language = 'English'
action = 'second_level_serve'
if action == 'initial_serve':
    if language == 'English':
        advisory_page = Page.objects.filter(title='ADVISORY_ENGLISH', parent_id=project_id)
    if language == 'Tamil':
        advisory_page = Page.objects.filter(title='ADVISORY_TAMIL',parent_id=project_id)

    child_page = Page.objects.filter(parent=advisory_page).order_by('_order')
    crop_names = child_page.values_list('title', flat=True)
    print(crop_names)
#     return Response(crop_names, status=status.HTTP_200_OK)
if action == 'second_level_serve':
    if language == 'English':
        advisory_page = Page.objects.filter(title='ADVISORY_ENGLISH', parent_id=project_id)
    if language == 'Tamil':
        advisory_page = Page.objects.filter(title='ADVISORY_TAMIL', parent_id=project_id)
    try:
        main_page = Page.objects.filter(title='Types', parent=advisory_page)
        print(main_page)
        advisory_type = Page.objects.filter(parent=main_page, title='Type1')
        print(advisory_type)
        first_child_page = Page.objects.filter(parent=advisory_type)
        print(first_child_page)
        advisory_type_ids = first_child_page.values_list('id', flat=True)
        print(advisory_type_ids)
        richtext_parent = RichTextPage.objects.filter(page_ptr_id__in=advisory_type_ids)
        richtext_parent_list = list(richtext_parent.values_list('id', 'parent', 'title', 'content'))
        richtext_parent_column = ['id', 'parent', 'parent_title', 'main_content']
        richtext_parent_df = pd.DataFrame(richtext_parent_list, columns=richtext_parent_column)
        second_child_page = Page.objects.filter(parent__in=first_child_page)
        second_child_page_ids = second_child_page.values_list('id', flat=True)
        richtext_child = RichTextPage.objects.filter(page_ptr_id__in=second_child_page_ids)
        richtext_child_list = list(
            richtext_child.values_list('page_ptr', 'page_ptr__title', 'parent', 'parent__title', 'content'))
        richtext_child_column = ['page_ptr', 'page_ptr_title', 'parent', 'title', 'content']
        richtext_child_df = pd.DataFrame(richtext_child_list, columns=richtext_child_column)
        final_df = richtext_parent_df.merge(richtext_child_df, how="outer", left_on='id', right_on='parent')
        final_df = final_df.fillna(0)
        master_df = final_df.merge(richtext_child_df, how="outer", left_on='page_ptr', right_on='page_ptr')
        master_df = master_df.fillna(0)
        indroduction_dict = {}
        for index, row in richtext_parent_df.iterrows():
            indroduction_dict[row['parent_title']] = row['main_content']
        master_dict = {}
        for index, row in master_df.iterrows():
            if row['parent_title'] not in master_dict:
                master_dict[row['parent_title']] = []
            temp_dict = {}
            if row['page_ptr_title_x'] != 0:
                temp_dict['title'] = row['page_ptr_title_x']
                temp_dict['content'] = row['content_x']
                temp_dict['page_id'] = row['page_ptr']
                master_dict[row['parent_title']].append(temp_dict)
        parent_child_dict = {
            'parent': indroduction_dict,
            'child': master_dict
        }
        print(parent_child_dict)
#         return Response(parent_child_dict, status=status.HTTP_200_OK)
    except Exception as e:
        print(e)

<SearchableQuerySet [<Page: Jamun / ADVISORY_ENGLISH / Types>]>
<SearchableQuerySet [<Page: Jamun / ADVISORY_ENGLISH / Types / Type1>]>
<SearchableQuerySet [<Page: Jamun / ADVISORY_ENGLISH / Types / Type1 / Question_type1>]>
<SearchableQuerySet [6]>
{'parent': {'Question_type1': '<p>Question_type1</p>'}, 'child': {'Question_type1': [{'title': 'Question1', 'content': '<p>Answer1</p>', 'page_id': 7}, {'title': 'Question2', 'content': '<p>Answer2</p>', 'page_id': 8}]}}


In [29]:
import pandas as pd
product_obj = Page.objects.filter(title='Products')
products = Page.objects.filter(parent=product_obj)
product_ids = products.values_list('id', flat=True)
richtext_parent = RichTextPage.objects.filter(page_ptr_id__in=product_ids)

richtext_parent_list = list(richtext_parent.values_list('id', 'parent', 'title', 'content'))
richtext_parent_column = ['id', 'parent', 'parent_title', 'main_content']
richtext_parent_df = pd.DataFrame(richtext_parent_list, columns=richtext_parent_column)
richtext_parent_df

second_parent = list(richtext_parent.values_list('id',flat=True))
second_richtext_parent = RichTextPage.objects.filter(parent_id__in=second_parent)

second_richtext_parent_list = list(second_richtext_parent.values_list('id', 'parent', 'title', 'content'))
second_richtext_parent_column = ['id', 'parent', 'parent_title', 'main_content']
second_richtext_parent_df = pd.DataFrame(second_richtext_parent_list, columns=second_richtext_parent_column)

data_dict = {}
data_dict['title'] = richtext_parent_df.to_dict('r')
data_dict['childs'] = second_richtext_parent_df.groupby('parent').apply(lambda x:x.to_dict('r')).to_dict()
data_dict

{'title': [{'id': 346,
   'parent': 332,
   'parent_title': 'Product 1',
   'main_content': '<p><img alt="" height="480" src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/640px-Image_created_with_a_mobile_phone.png" width="640"></p>'},
  {'id': 348,
   'parent': 332,
   'parent_title': 'product 2',
   'main_content': '<p>https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/640px-Image_created_with_a_mobile_phone.png</p>'}],
 'childs': {346: [{'id': 347,
    'parent': 346,
    'parent_title': 'Intro',
    'main_content': '<p>Images may be two or three-<a class="mw-redirect" href="https://en.wikipedia.org/wiki/Dimensional" title="Dimensional">dimensional</a>, such as a<span>&nbsp;</span><a href="https://en.wikipedia.org/wiki/Photograph" title="Photograph">photograph</a><span>&nbsp;</span>or screen display, or three-dimensional, such as a<span>&nbsp;</span><a href="https://en.wikipedia.org/wiki/S

In [23]:

second_parent
RichTextPage.objects.filter(page_ptr_id__in=second_parent)

[346, 348]

In [2]:
import pandas as pd
product_obj = Page.objects.filter(title='Products')
products = Page.objects.filter(parent=product_obj)
product_ids = products.values_list('id', flat=True)
richtext_parent = RichTextPage.objects.filter(page_ptr_id__in=product_ids)

richtext_parent_list = list(richtext_parent.values_list('id', 'parent', 'title', 'content'))
richtext_parent_column = ['id', 'parent', 'parent_title', 'main_content']
richtext_parent_df = pd.DataFrame(richtext_parent_list, columns=richtext_parent_column)
richtext_parent_df

second_parent = list(richtext_parent.values_list('id',flat=True))
second_richtext_parent = RichTextPage.objects.filter(parent_id__in=second_parent)
pd.DataFrame(list(second_richtext_parent.values_list('parent_id','parent__title', 'description', 'parent__description')), columns=['id', 'title', 'description', 'image']).to_dict('r')

/home/r_k_vijay/Softwares/virtualenv/pagems/lib/python3.9/site-packages/pandas/core/frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


[{'id': 346,
  'title': 'Product 1',
  'description': 'Images may be two or three-dimensional, such as a&nbsp;photograph&nbsp;or screen display, or three-dimensional, such as a&nbsp;statue&nbsp;or&nbsp;hologram. They may be captured by&nbsp;optical&nbsp;devices &ndash; such as&nbsp;cameras,&nbsp;mirrors,&nbsp;lenses,&nbsp;telescopes,&nbsp;microscopes, etc. and natural objects and phenomena, such as the&nbsp;human eye&nbsp;or water.',
  'image': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Image_created_with_a_mobile_phone.png/640px-Image_created_with_a_mobile_phone.png'},
 {'id': 348,
  'title': 'product 2',
  'description': 'Images may be two or three-dimensional, such as a&nbsp;photograph&nbsp;or screen display, or three-dimensional, such as a&nbsp;statue&nbsp;or&nbsp;hologram. They may be captured by&nbsp;optical&nbsp;devices &ndash; such as&nbsp;cameras,&nbsp;mirrors,&nbsp;lenses,&nbsp;telescopes,&nbsp;microscopes, etc. and natural objects and phenomena, such as the&n

In [34]:
from base64 import b64encode, b64decode
from django.conf import settings

In [43]:
def advisory_image_location(image_tag):
    try:
        split1 = image_tag.split("src=")[1]
        print(split1)
        split2 = split1.split(" ")[:-1]
        print(split2)
        print(str(split2)[9:-3])
        return str(split2)[9:-3]
    except Exception as e:
        print(e)
        return ""

In [44]:
def encode_image_to_base64(image_path):
    try:
        image_path = settings.MEDIA_ROOT + image_path
        with open(image_path, 'rb') as image_file:
            encoded_image = b64encode(image_file.read())
            return encoded_image
    except Exception as e:
        print(e)
        return ""

In [45]:
advisory_page = Page.objects.filter(title='ADVISORY_TAMIL',parent_id=1)

child_page = Page.objects.filter(parent=advisory_page).order_by('_order')
crop_names = child_page.values_list('title', 'richtextpage__content', 'id')
crop_columns = ['title', 'image_link', 'id']
crop_df = pd.DataFrame(crop_names, columns=crop_columns)

crop_df['image_link'] = crop_df.apply(lambda x: advisory_image_location(x['image_link']), axis=1)
crop_df['image'] = crop_df.apply(lambda x: encode_image_to_base64(x['image_link']), axis=1)

parent_ids = list(child_page.values_list('id', flat=True))
crop_level_two_obj = Page.objects.filter(parent_id__in=parent_ids).order_by('_order')
crop_level_two_list = list(crop_level_two_obj.values_list('id', 'title', 'parent_id'))
crop_level_two_column = ['id', 'title', 'parent_id']
crop_level_two_df = pd.DataFrame(crop_level_two_list, columns=crop_level_two_column)
crop_level_two_dict = crop_level_two_df.groupby('parent_id').apply(lambda x:x.to_dict('r')).to_dict()
data_dict = {
    'crop_list': crop_df.to_dict('r'),
    'crop_dict': crop_level_two_dict
 }
data_dict

"/media/uploads/wri_te.jpeg" width="299"></p>
['"/media/uploads/wri_te.jpeg"']
/uploads/wri_te.jpeg
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
[Errno 21] Is a directory: '/home/sunesh/Software/page_ms/page_microservice/media'
[Errno 21] Is a directory: '/home/sunesh/Software/page_ms/page_microservice/media'
[Errno 21] Is a directory: '/home/sunesh/Software/page_ms/page_microservice/media'
[Errno 21] Is a directory: '/home/sunesh/Software/page_ms/page_microservice/media'
[Errno 21] Is a directory: '/home/sunesh/Software/page_ms/page_microservice/media'
[Errno 21] Is a directory: '/home/sunesh/Software/page_ms/page_microservice/media'
[Errno 21] Is a directory: '/home/sunesh/Software/page_ms/page_microservice/media'
[Errno 21] Is a directory: '/home/sunesh/Software/page_ms/page_microservice/med

{'crop_list': [{'title': 'சவுக்கு',
   'image_link': '/uploads/wri_te.jpeg',
   'id': 200,
   'image': b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxQTEhUTExMWFRUXGB0YFxgYFRgYGBgYGBUXGhgVFx0dHSggGB0lHx4XITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGxAQGy0mHSUtLS0tLSstLS8tLS0tLS0tLS0tNS0tLTUtLi0tLS0tLTIrLS0tLS0tLS0tLS0tLS0tLf/AABEIAKgBKwMBIgACEQEDEQH/xAAcAAEBAAIDAQEAAAAAAAAAAAAAAQQGAwUHAgj/xAA9EAABAgMEBwYFBAICAQUAAAABAhEAITEDBBJBEyIyUWFxgQUGI5GhwTNC0fDxB0NjsWLhFFJyFTRTgrL/xAAZAQEAAwEBAAAAAAAAAAAAAAAAAQIDBAX/xAAjEQEAAgIDAQACAgMAAAAAAAAAAQIDERIhMUEEMlFxE0LR/9oADAMBAAIRAxEAPwD2Za9JISac4Y2GjzpwnBbft1zbd9tCTf5+r5QBCtHIzfdESjAcRod3GKhv3K5PES76+zx9IAUOdJlXjL8QX4lJNv4wLvLY9Gz94L/j6t6e8BVLxDAK+koJXhGA19JwUzau368YJZtba9eEBEDR1m+7hAIY6TKvGf5gj+To/rAO89j0bL2gC0YziEhScVatJISac4i3+TZ4b4q2/brm26AY5aPOj5QQvRyM3nKEm/z9Xghv3K5PugIhGjOIzeUvP2gUOdJlXjKCH/cpx3/bwLvLY9Gz94AsaSkm3xVLxjCKjfSURf8AH1aKpm1NrNvWABeEYDWnCcRHh1m+7h+YoZtbb9XyiI/k6P6+0ACMJxmleM4KRjOIU41lAO+tsejZQU76mzw9YCrOkpJt8Ctxo86PlKC/4+rQLNLb9Xz94Ahej1TN5y8oiE6